<a href="https://colab.research.google.com/github/ashirm1999/Prudential-Life-Insurance-Assessment/blob/main/Medium_Risk_Level_ML_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import recall_score, precision_score, accuracy_score, f1_score, roc_auc_score 
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_halving_search_cv 
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA
pd.options.display.max_columns = 500

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Northeastern Projects/Capstone Data/Features_Data.csv')

In [ ]:
df

,Product_Info_1,Product_Info_2,Product_Info_3,Product_Info_4,Product_Info_5,Product_Info_6,Product_Info_7,Ins_Age,Ht,Wt,BMI,Employment_Info_1,Employment_Info_2,Employment_Info_3,Employment_Info_4,Employment_Info_5,Employment_Info_6,InsuredInfo_1,InsuredInfo_2,InsuredInfo_3,InsuredInfo_4,InsuredInfo_5,InsuredInfo_6,InsuredInfo_7,Insurance_History_1,Insurance_History_2,Insurance_History_3,Insurance_History_4,Insurance_History_5,Insurance_History_7,Insurance_History_8,Insurance_History_9,Family_Hist_1,Family_Hist_2,Family_Hist_4,Medical_History_1,Medical_History_2,Medical_History_3,Medical_History_4,Medical_History_5,Medical_History_6,Medical_History_7,Medical_History_8,Medical_History_9,Medical_History_11,Medical_History_12,Medical_History_13,Medical_History_14,Medical_History_16,Medical_History_17,Medical_History_18,Medical_History_19,Medical_History_20,Medical_History_21,Medical_History_22,Medical_History_23,Medical_History_25,Medical_History_26,Medical_History_27,Medical_History_28,Medical_History_29,Medical_History_30,Medical_History_31,Medical_History_33,Medical_History_34,Medical_History_35,Medical_History_36,Medical_History_37,Medical_History_38,Medical_History_39,Medical_History_40,Medical_History_41,Medical_Keyword_1,Medical_Keyword_2,Medical_Keyword_3,Medical_Keyword_4,Medical_Keyword_5,Medical_Keyword_6,Medical_Keyword_7,Medical_Keyword_8,Medical_Keyword_9,Medical_Keyword_10,Medical_Keyword_11,Medical_Keyword_12,Medical_Keyword_13,Medical_Keyword_14,Medical_Keyword_15,Medical_Keyword_16,Medical_Keyword_17,Medical_Keyword_18,Medical_Keyword_19,Medical_Keyword_20,Medical_Keyword_21,Medical_Keyword_22,Medical_Keyword_23,Medical_Keyword_24,Medical_Keyword_25,Medical_Keyword_26,Medical_Keyword_27,Medical_Keyword_28,Medical_Keyword_29,Medical_Keyword_30,Medical_Keyword_31,Medical_Keyword_32,Medical_Keyword_33,Medical_Keyword_34,Medical_Keyword_35,Medical_Keyword_36,Medical_Keyword_37,Medical_Keyword_38,Medical_Keyword_39,Medical_Keyword_40,Medical_Keyword_41,Medical_Keyword_42,Medical_Keyword_43,Medical_Keyword_44,Medical_Keyword_45,Medical_Keyword_46,Medical_Keyword_47,Medical_Keyword_48,Response,new_response,Product_Info_2_num,Product_Info_2_char,BMI_x_Age,Ht_x_Age,Wt_x_Age,BMI_Wt_category,Age_category,Height_category,Weight_category,Total_Medical_Keywords_Count,overall_risk,average_risk,low_risk,high_risk
0,0,16,8,0.076923,0,0,0,0.641791,0.581818,0.148536,0.323008,0.028,10,0,0.000000,1,0.361469,0,0,5,1,0,1,0,0,0,2,0,0.000667,0,0,1,1,0.474550,0.444890,4.0,102,1,0,0,2,1,1,0,2,1,2,2,2,2,0,0,1,0,1,2,0,2,2,0,2,1,2,0,2,0,1,1,0,2,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,2,2,3,0.207304,0.373406,0.095329,2,1,1,2,0,0,1,0,0
1,0,0,22,0.076923,0,1,0,0.059701,0.600000,0.131799,0.272288,0.000,0,1,0.000000,0,0.001800,0,0,5,1,0,1,0,1,0,2,0,0.000133,0,2,1,1,0.188406,0.084507,5.0,368,1,0,0,2,1,1,0,2,1,2,2,0,2,0,0,1,0,1,2,0,2,2,0,2,1,2,1,0,0,1,1,0,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0.016256,0.035821,0.007869,2,2,1,2,0,1,1,0,0
2,0,18,22,0.076923,0,1,0,0.029851,0.745455,0.288703,0.428780,0.030,7,0,0.000000,0,0.030000,0,0,7,1,0,0,0,1,0,0,2,0.001733,2,1,2,2,0.304348,0.225352,10.0,2,1,1,0,2,1,1,1,2,1,2,2,0,2,0,0,1,0,1,2,1,1,2,0,2,1,2,1,2,0,2,1,0,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,2,0,4,0.012799,0.022252,0.008618,0,2,0,0,0,1,0,0,1
3,0,17,8,0.487179,0,1,0,0.164179,0.672727,0.205021,0.352438,0.042,7,0,0.000000,1,0.200000,1,0,7,1,0,1,0,1,0,0,2,0.001733,2,1,2,2,0.420290,0.352113,0.0,312,1,1,0,2,1,1,1,2,1,2,2,0,2,0,0,1,1,1,2,0,2,2,0,2,1,2,1,2,0,1,1,0,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,2,3,3,0.057863,0.110448,0.033660,2,2,0,2,1,1,0,0,0
4,0,15,22,0.230769,0,1,0,0.417910,0.654545,0.234310,0.424046,0.027,7,0,0.000000,0,0.050000,0,0,5,1,0,1,0,1,0,0,2,0.001733,2,1,2,1,0.463768,0.408451,7.

In [ ]:
medical_columns = df.columns[df.columns.str.startswith('Medical_Keyword_')]

In [ ]:
df.drop(medical_columns, axis = 1, inplace = True)

In [ ]:
df

,Product_Info_1,Product_Info_2,Product_Info_3,Product_Info_4,Product_Info_5,Product_Info_6,Product_Info_7,Ins_Age,Ht,Wt,BMI,Employment_Info_1,Employment_Info_2,Employment_Info_3,Employment_Info_4,Employment_Info_5,Employment_Info_6,InsuredInfo_1,InsuredInfo_2,InsuredInfo_3,InsuredInfo_4,InsuredInfo_5,InsuredInfo_6,InsuredInfo_7,Insurance_History_1,Insurance_History_2,Insurance_History_3,Insurance_History_4,Insurance_History_5,Insurance_History_7,Insurance_History_8,Insurance_History_9,Family_Hist_1,Family_Hist_2,Family_Hist_4,Medical_History_1,Medical_History_2,Medical_History_3,Medical_History_4,Medical_History_5,Medical_History_6,Medical_History_7,Medical_History_8,Medical_History_9,Medical_History_11,Medical_History_12,Medical_History_13,Medical_History_14,Medical_History_16,Medical_History_17,Medical_History_18,Medical_History_19,Medical_History_20,Medical_History_21,Medical_History_22,Medical_History_23,Medical_History_25,Medical_History_26,Medical_History_27,Medical_History_28,Medical_History_29,Medical_History_30,Medical_History_31,Medical_History_33,Medical_History_34,Medical_History_35,Medical_History_36,Medical_History_37,Medical_History_38,Medical_History_39,Medical_History_40,Medical_History_41,Response,new_response,Product_Info_2_num,Product_Info_2_char,BMI_x_Age,Ht_x_Age,Wt_x_Age,BMI_Wt_category,Age_category,Height_category,Weight_category,Total_Medical_Keywords_Count,overall_risk,average_risk,low_risk,high_risk
0,0,16,8,0.076923,0,0,0,0.641791,0.581818,0.148536,0.323008,0.028,10,0,0.000000,1,0.361469,0,0,5,1,0,1,0,0,0,2,0,0.000667,0,0,1,1,0.474550,0.444890,4.0,102,1,0,0,2,1,1,0,2,1,2,2,2,2,0,0,1,0,1,2,0,2,2,0,2,1,2,0,2,0,1,1,0,2,2,2,8,2,2,3,0.207304,0.373406,0.095329,2,1,1,2,0,0,1,0,0
1,0,0,22,0.076923,0,1,0,0.059701,0.600000,0.131799,0.272288,0.000,0,1,0.000000,0,0.001800,0,0,5,1,0,1,0,1,0,2,0,0.000133,0,2,1,1,0.188406,0.084507,5.0,368,1,0,0,2,1,1,0,2,1,2,2,0,2,0,0,1,0,1,2,0,2,2,0,2,1,2,1,0,0,1,1,0,2,2,0,4,0,0,0,0.016256,0.035821,0.007869,2,2,1,2,0,1,1,0,0
2,0,18,22,0.076923,0,1,0,0.029851,0.745455,0.288703,0.428780,0.030,7,0,0.000000,0,0.030000,0,0,7,1,0,0,0,1,0,0,2,0.001733,2,1,2,2,0.304348,0.225352,10.0,2,1,1,0,2,1,1,1,2,1,2,2,0,2,0,0,1,0,1,2,1,1,2,0,2,1,2,1,2,0,2,1,0,2,2,0,8,2,0,4,0.012799,0.022252,0.008618,0,2,0,0,0,1,0,0,1
3,0,17,8,0.487179,0,1,0,0.164179,0.672727,0.205021,0.352438,0.042,7,0,0.000000,1,0.200000,1,0,7,1,0,1,0,1,0,0,2,0.001733,2,1,2,2,0.420290,0.352113,0.0,312,1,1,0,2,1,1,1,2,1,2,2,0,2,0,0,1,1,1,2,0,2,2,0,2,1,2,1,2,0,1,1,0,2,2,0,8,2,3,3,0.057863,0.110448,0.033660,2,2,0,2,1,1,0,0,0
4,0,15,22,0.230769,0,1,0,0.417910,0.654545,0.234310,0.424046,0.027,7,0,0.000000,0,0.050000,0,0,5,1,0,1,0,1,0,0,2,0.001733,2,1,2,1,0.463768,0.408451,7.0,148,1,1,0,2,1,1,1,2,1,2,2,0,2,0,0,1,0,1,2,1,1,2,0,2,1,2,1,2,0,2,1,0,2,2,0,8,2,1,3,0.177213,0.273541,0.097920,0,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59376,0,14,8,0.230769,0,1,0,0.074627,0.709091,0.320084,0.519103,0.020,0,1,0.000000,1,0.025000,0,0,7,1,0,1,0,1,0,0,2,0.001733,2,1,2,2,0.217391,0.197183,0.0,237,1,0,0,2,1,1,1,2,1,2,2,0,2,0,0,1,0,1,2,0,2,2,0,2,1,2,1,2,0,1,1,0,2,2,2,4,0,0,3,0.038739,0.052917,0.023887,0,2,0,0,0,1,0,0,1
59377,0,16,22,0.230769,0,1,0,0.432836,0.800000,0.403766,0.551119,0.100,7,0,0.000010,0,0.350000,0,0,2,1,0,0,0,1,0,2,1,0.000267,0,2,1,2,0.565217,0.478873,24.0,436,1,1,0,2,1,1,1,2,1,2,2,0,2,0,0,1,0,1,2,1,1,2,0,2,1,2,1,2,0,2,1,0,2,2,0,7,1,2,3,0.238544,0.346269,0.174764,1,0,2,1,0,0,0,1,0
59378,0,18,22,0.076923,0,1,0,0.104478,0.745455,0.246862,0.360969,0.035,7,0,0.000000,0,0.361469,0,0,5,1,0,0,0,1,0,0,2,0.001733,2,1,2,2,0.173913,0.126761,7.0,148,1,1,0,2,1,1,1,2,1,2,2,0,2,0,0,1,0,1,2,1,1,2,0,2,1,2,1,0,0,2,1,0,2,2,0,8,2,0,

In [ ]:
X = df.drop(['Response', 'new_response'], axis = 1)
y1 = df['new_response']
y2 = df['Response']

# Results Section

In [ ]:
def evaluate_model(actual, predicted):
  '''
  Evaluates the performance of a classification model by calculating and printing the recall, precision, accuracy, F1 score and ROC AUC score.
    
  Args:
    actual - true class labels of the target variable
    predicted - predicted class labels of the target variable
    
  Returns:
    None
  '''
  # Calculate recall score
  recall = recall_score(actual, predicted, average = 'macro')

  # Calculate precision score
  precision = precision_score(actual, predicted, average = 'macro')

  # Calculate accuracy score
  accuracy = accuracy_score(actual, predicted)

  # Calculate F1 score
  f1 = f1_score(actual, predicted, average = 'macro')

  return round(recall, 3), round(precision, 3), round(accuracy, 3), round(f1, 3)

In [ ]:
def fit_classifiers(data, target, test_size = 0.2):
  '''
  This Function fits the given data using multiple Models and calculates all the 5 evaluation metrics and stores the result in form of dataframe 
    
  Args:
    data - Contains the Input data consisting all the features 
    target - the Target variables
    test_size - default value 0.2
    
  Returns:
    df - dataframe containing the results of multiple models along with all the evalution metrics
    train_data - Training Data used for training the model
    val_data - Validation Data used for validating the results
    test_data - Testing Data used for testing the model
    train_labels - Training Target Variable
    val_labels - Validation Target Variable
    test_labels - Testing Target Variable
  '''

  # splitting data into training (70%) and testing (30%) sets
  train_data, test_data, train_labels, test_labels = train_test_split(data, target, test_size = 0.3, random_state = 42, stratify = target)

  # splitting the training set further into training (60%) and validation (40%) sets
  train_data, val_data, train_labels, val_labels = train_test_split(train_data, train_labels, test_size = 0.4, random_state = 42, stratify = train_labels)
  
  # Defining the models in form of dictonary where keys are the model name and values are the model function
  models = {
        'XGBClassifier': XGBClassifier(),
        'RandomForestClassifier': RandomForestClassifier(),
        'LogisticRegression': LogisticRegression(max_iter = 200),
        'KNeighborsClassifier': KNeighborsClassifier(),
        'CatBoost': CatBoostClassifier(iterations = 200),
        'GaussianNB': GaussianNB(),
        'SVM': SVC() }
    
  results = []

  # Iterating over each model
  for name, model in models.items():

    # Fitting the model with the given training data
    model.fit(train_data, train_labels)
    
    # Predicting the Training Data
    y_train_pred = model.predict(train_data)

    # Predicting the Testing Data
    y_val_pred = model.predict(val_data)
    
    # Calculating the Training Evaluation Metricx
    train_recall, train_precision, train_acc, train_f1 = evaluate_model(train_labels, y_train_pred)

    # Calculating the Testing Evaluation Metrics
    val_recall, val_precision, val_acc, val_f1 = evaluate_model(val_labels, y_val_pred)

    # Storing the entire result as a row
    result = [name,
              train_recall,
              train_precision,
              train_acc,
              train_f1,
              val_recall,
              val_precision,
              val_acc,
              val_f1]

    # Appending the row into the master result to form the dataframe   
    results.append(result)
  
  # Forming the dataframe with the result
  df = pd.DataFrame(results, columns = ['Classifier', 'Train Recall', 'Train Precision', 'Train Accuracy', 'Train F1 Score', 'Val Recall', 'Val Precision', 'Val Accuracy', 'Val F1 Score'])
    
  return df, train_data, val_data, test_data, train_labels, val_labels, test_labels

## Results for New Target Variable i.e having 3 binned classes

In [ ]:
# Running the model pipeline for the new target variable
output, Xtrain, Xval, Xtest, ytrain, yval, ytest = fit_classifiers(X, y1)

In [ ]:
output

,Classifier,Train Recall,Train Precision,Train Accuracy,Train F1 Score,Val Recall,Val Precision,Val Accuracy,Val F1 Score
0,XGBClassifier,0.641,0.670,0.665,0.639,0.631,0.656,0.656,0.628
1,RandomForestClassifier,1.000,1.000,1.000,1.000,0.618,0.642,0.644,0.616
2,LogisticRegression,0.552,0.578,0.580,0.550,0.551,0.577,0.579,0.549
3,KNeighborsClassifier,0.626,0.642,0.639,0.631,0.432,0.442,0.448,0.435
4,CatBoost,0.733,0.753,0.747,0.737,0.643,0.654,0.660,0.643
5,GaussianNB,0.527,0.534,0.527,0.503,0.522,0.527,0.522,0.498
6,SVM,0.359,0.485,0.435,0.270,0.358,0.528,0.434,0.270


In [ ]:
# Running the Best Performing Model
best_model = CatBoostClassifier(iterations = 200)
best_model.fit(Xtrain, ytrain)
ypred = best_model.predict(Xval)
print(classification_report(yval, ypred))

Learning rate set to 0.36324
0:	learn: 0.9646523	total: 98ms	remaining: 19.5s
1:	learn: 0.9022874	total: 209ms	remaining: 20.7s
2:	learn: 0.8633194	total: 288ms	remaining: 18.9s
3:	learn: 0.8430522	total: 394ms	remaining: 19.3s
4:	learn: 0.8246455	total: 487ms	remaining: 19s
5:	learn: 0.8146935	total: 574ms	remaining: 18.6s
6:	learn: 0.8063655	total: 643ms	remaining: 17.7s
7:	learn: 0.7964021	total: 722ms	remaining: 17.3s
8:	learn: 0.7925644	total: 796ms	remaining: 16.9s
9:	learn: 0.7877212	total: 849ms	remaining: 16.1s
10:	learn: 0.7846921	total: 892ms	remaining: 15.3s
11:	learn: 0.7812406	total: 925ms	remaining: 14.5s
12:	learn: 0.7762638	total: 964ms	remaining: 13.9s
13:	learn: 0.7720843	total: 993ms	remaining: 13.2s
14:	learn: 0.7677953	total: 1.04s	remaining: 12.9s
15:	learn: 0.7641444	total: 1.08s	remaining: 12.4s
16:	learn: 0.7627389	total: 1.11s	remaining: 12s
17:	learn: 0.7606365	total: 1.16s	remaining: 11.7s
18:	learn: 0.7580331	total: 1.19s	remaining: 11.4s
19:	learn: 0.7559

## Results for Old Target Variable i.e having 8 classes

In [ ]:
# Running the model pipeline for the old target variable
output1, Xtrain1, Xval1, Xtest1, ytrain1, yval1, ytest1 = fit_classifiers(X, y2)

In [ ]:
output1

,Classifier,Train Recall,Train Precision,Train Accuracy,Train F1 Score,Val Recall,Val Precision,Val Accuracy,Val F1 Score
0,XGBClassifier,0.370,0.527,0.546,0.384,0.350,0.478,0.529,0.356
1,RandomForestClassifier,1.000,1.000,1.000,1.000,0.333,0.471,0.517,0.338
2,LogisticRegression,0.207,0.256,0.399,0.194,0.206,0.242,0.397,0.192
3,KNeighborsClassifier,0.395,0.474,0.521,0.413,0.177,0.179,0.294,0.175
4,CatBoost,0.560,0.679,0.660,0.600,0.370,0.425,0.526,0.384
5,GaussianNB,0.309,0.281,0.374,0.258,0.295,0.274,0.370,0.249
6,SVM,0.125,0.041,0.328,0.062,0.125,0.041,0.328,0.062


In [ ]:
# Running the Best Performing Model
best_model1 = CatBoostClassifier(iterations = 200)
best_model1.fit(Xtrain1, ytrain1)
ypred1 = best_model1.predict(Xval1)
print(classification_report(yval1, ypred1))

Learning rate set to 0.36324
0:	learn: 1.6333615	total: 133ms	remaining: 26.5s
1:	learn: 1.5441049	total: 243ms	remaining: 24s
2:	learn: 1.4828320	total: 328ms	remaining: 21.5s
3:	learn: 1.4539738	total: 426ms	remaining: 20.9s
4:	learn: 1.4308466	total: 532ms	remaining: 20.7s
5:	learn: 1.4102392	total: 612ms	remaining: 19.8s
6:	learn: 1.3926187	total: 692ms	remaining: 19.1s
7:	learn: 1.3728527	total: 780ms	remaining: 18.7s
8:	learn: 1.3618771	total: 873ms	remaining: 18.5s
9:	learn: 1.3551693	total: 959ms	remaining: 18.2s
10:	learn: 1.3413700	total: 1.07s	remaining: 18.4s
11:	learn: 1.3344257	total: 1.16s	remaining: 18.1s
12:	learn: 1.3282810	total: 1.23s	remaining: 17.7s
13:	learn: 1.3240797	total: 1.32s	remaining: 17.6s
14:	learn: 1.3197435	total: 1.41s	remaining: 17.4s
15:	learn: 1.3164885	total: 1.48s	remaining: 17s
16:	learn: 1.3101372	total: 1.56s	remaining: 16.8s
17:	learn: 1.3051203	total: 1.64s	remaining: 16.6s
18:	learn: 1.3001480	total: 1.73s	remaining: 16.4s
19:	learn: 1.296

# Grid Search Optimization

In [ ]:
def grid_search_best_param(model, Xtrain, ytrain, param_grid):
  '''
  The function implements the Grid Search CV using the given model, data and parameters.
    
  Args:
    model - the model on which grid search will be implemented
    Xtrain - training data used for optimizing the model
    ytrain - training labels
    param_grid - dictonary containing different values to be test for optimization
    
  Returns:
    best_params - the best parameters for which the model achieved highest accuracy
  '''
  
  model_cv = HalvingGridSearchCV(estimator = model, param_grid = param_grid)
  model_cv.fit(Xtrain, ytrain)
  best_params = model_cv.best_params_

  return best_params

In [ ]:
def model_tuning(Xtrain, ytrain, Xval, yval):
  '''
  The function implements the Grid Search CV for the top 3 best models
    
  Args:
    Xtrain - training data used for optimizing the model
    ytrain - training labels
    Xval - testing data used for optimizing the model
    yval - testing labels
    
  Returns:
    result_df - dataframe containing the results of top 3 models along after optimization
  '''

  result = []

  models = {
        'CatBoost': CatBoostClassifier(iterations = 200, verbose = False),
        'XGBClassifier': XGBClassifier(),
        'RandomForestClassifier': RandomForestClassifier()}

  for name, model in models.items():
    if name == 'RandomForestClassifier':
      param_grid = { 'n_estimators': [200, 500],
                    'max_features': ['auto', 'sqrt', 'log2'],
                    'max_depth' : [4,5,6,7,8],
                    'criterion' :['gini', 'entropy']}

      params = grid_search_best_param(model, Xtrain, ytrain, param_grid)
      model_final = RandomForestClassifier(max_features = params['max_features'], n_estimators = params['n_estimators'], max_depth = params['max_depth'], criterion = params['criterion'])
      model_final.fit(Xtrain, ytrain)
      y_val_pred = model_final.predict(Xval)
      val_recall, val_precision, val_acc, val_f1 = evaluate_model(yval, y_val_pred)

      row = [str(name), val_recall, val_precision, val_acc, val_f1, params]
      print('RandomForest')
      print(row)
      result.append(row)

    elif name == 'XGBClassifier':
      param_grid = { 'min_child_weight': [1, 5, 10],
                    'gamma': [0.5, 1, 1.5, 2, 5],
                    'subsample': [0.6, 0.8, 1.0],
                    'colsample_bytree': [0.6, 0.8, 1.0],
                    'max_depth': [3, 4, 5] }

      params = grid_search_best_param(model, Xtrain, ytrain, param_grid)
      model_final = XGBClassifier(min_child_weight = params['min_child_weight'], gamma = params['gamma'], subsample = params['subsample'], colsample_bytree = params['colsample_bytree'], max_depth = params['max_depth'])
      model_final.fit(Xtrain, ytrain)
      y_val_pred = model_final.predict(Xval)
      val_recall, val_precision, val_acc, val_f1 = evaluate_model(yval, y_val_pred)

      row = [str(name), val_recall, val_precision, val_acc, val_f1, params]
      print('XGBoost')
      print(row)
      result.append(row)

    elif name == 'CatBoost':
      param_grid = {'depth': [4, 5, 6],
                    'leaf_estimation_iterations': [10, 20]}

      params = grid_search_best_param(model, Xtrain, ytrain, param_grid)

      model_final = CatBoostClassifier(iterations = 200, loss_function = 'MultiClass', depth = params['depth'], leaf_estimation_iterations = params['leaf_estimation_iterations'], verbose = False)
      model_final.fit(Xtrain, ytrain)
      y_val_pred = model_final.predict(Xval)
      val_recall, val_precision, val_acc, val_f1 = evaluate_model(yval, y_val_pred)

      row = [str(name), val_recall, val_precision, val_acc, val_f1, params]
      print('CatBoost')
      print(row)
      result.append(row)
    
    else:
      pass


  result_df = pd.DataFrame(result, columns = ['Classifer', 'Val Recall', 'Val Precision', 'Val Accuracy', 'Val F1 Score', 'Parameters'])
  return result_df

XGBoost
['XGBClassifier', 0.638, 0.662, 0.662, 0.637, {'colsample_bytree': 0.8, 'gamma': 2, 'max_depth': 4, 'min_child_weight': 1, 'subsample': 1.0}]
RandomForest
['RandomForestClassifier', 0.583, 0.661, 0.625, 0.568, {'criterion': 'gini', 'max_depth': 8, 'max_features': 'auto', 'n_estimators': 200}]

## Results for New Target Variable i.e having 3 binned classes

In [ ]:
# Optimizing the Model for the new target variable
output_cv1 = model_tuning(Xtrain, ytrain, Xval, yval)

CatBoost
['CatBoost', 0.629, 0.661, 0.655, 0.628, {'depth': 6, 'leaf_estimation_iterations': 10}]
XGBoost
['XGBClassifier', 0.641, 0.66, 0.662, 0.64, {'colsample_bytree': 1.0, 'gamma': 5, 'max_depth': 5, 'min_child_weight': 5, 'subsample': 0.6}]
RandomForest
['RandomForestClassifier', 0.582, 0.659, 0.623, 0.566, {'criterion': 'gini', 'max_depth': 8, 'max_features': 'sqrt', 'n_estimators': 200}]


In [ ]:
output_cv1

,Classifer,Val Recall,Val Precision,Val Accuracy,Val F1 Score,Parameters
0,CatBoost,0.629,0.661,0.655,0.628,"{'depth': 6, 'leaf_estimation_iterations': 10}"
1,XGBClassifier,0.641,0.660,0.662,0.640,"{'colsample_bytree': 1.0, 'gamma': 5, 'max_dep..."
2,RandomForestClassifier,0.582,0.659,0.623,0.566,"{'criterion': 'gini', 'max_depth': 8, 'max_fea..."


## Results for Old Target Variable i.e having 8 classes

In [ ]:
# Optimizing the Model for the old target variable
output_cv2 = model_tuning(Xtrain1, ytrain1, Xval1, yval1)

CatBoost
['CatBoost', 0.334, 0.474, 0.521, 0.34, {'depth': 6, 'leaf_estimation_iterations': 20}]
XGBoost
['XGBClassifier', 0.366, 0.462, 0.538, 0.378, {'colsample_bytree': 0.8, 'gamma': 2, 'max_depth': 4, 'min_child_weight': 5, 'subsample': 0.8}]
RandomForest
['RandomForestClassifier', 0.295, 0.353, 0.496, 0.285, {'criterion': 'gini', 'max_depth': 8, 'max_features': 'sqrt', 'n_estimators': 500}]


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
output_cv2

,Classifer,Val Recall,Val Precision,Val Accuracy,Val F1 Score,Parameters
0,CatBoost,0.334,0.474,0.521,0.340,"{'depth': 6, 'leaf_estimation_iterations': 20}"
1,XGBClassifier,0.366,0.462,0.538,0.378,"{'colsample_bytree': 0.8, 'gamma': 2, 'max_dep..."
2,RandomForestClassifier,0.295,0.353,0.496,0.285,"{'criterion': 'gini', 'max_depth': 8, 'max_fea..."
